In [1]:
import numpy as np
from setup_mnist import *


Using TensorFlow backend.


In [2]:
data = MNIST()
X_exp = np.load("multiclass_data_2/X_exp.npy")
Y_exp = np.load("multiclass_data_2/Y_exp.npy")

In [ ]:
X_exp.shape

In [3]:
X_exp = X_exp.reshape(-1, 28, 28, 1)
Y_exp = np.array([(np.arange(10) == l).astype(np.float32) for l in Y_exp])

In [4]:
model_dir = "deep_networks"
models = [conv_net(1, model_dir + "/conv1"), conv_net(0, model_dir + "/conv2"), 
          multilayer(4, 128, model_dir + "/mlp1"), multilayer(2, 256, model_dir + "/mlp2"),
          multilayer(0, 0, model_dir + "/zero_layer")]

In [5]:
[model.evaluate(X_exp, Y_exp)[1] for model in models]

100/100 [==============================] - 0s 265us/step


[1.0, 1.0, 0.99, 0.99, 1.0]

In [7]:
np.linalg.norm(np.ones((2,2)).reshape(-1,))

2.0